# Recommender Systems Tutorial

This notebook will give you a short overview on how to implement different recommender systems approaches. We will start with implementing a basic `k nearest neighbors`(KNN) approach from scratch and will then continue to implement a movie recommender based on `sklearn`'s nearest neighbor method and finally we will compare two factorization approaches using an out-of-the-box recommendation framework.

## Setup

Later in this tutorial, we will use a library called Surprise. This can be installed via pip.

In [1]:
! pip install scikit-surprise

Throughout the entire tutorial, we will use `numpy` and `pandas` as well as some `scipy` and `sklearn` functions. If you haven't installed these yet, now might be the time.

In [2]:
import numpy as np
import pandas as pd

## Neighborhood-Based Methods

We start the tutorial with an illustrative example matrix that consists four users and 5 items. This is the same matrix that was used in the slide to this lecture.

<img src="images/explicit-matrix.png" width=400 />

In [3]:
example_matrix = np.array(
    [
        [5, 1, 0, 2, 2],
        [1, 5, 2, 5, 5],
        [2, 0, 3, 5, 4],
        [4, 3, 5, 3, 0]
    ]
)

Let's try to predict how John would rate _Die Hard_. First, we need to find the users that are closest to John. There are multiple similarity or distance functions that can be applied. In this case, we will use the **cosine similarity**. We treat each user's rating behavior as a vector and compare them with each other. The result will be a number between 0 and 1, with 1 being two identical vectors.

In [4]:
def cosine_similarity(xu, xv):
    return np.dot(xu, xv)/(np.linalg.norm(xu)*np.linalg.norm(xv))

Let's compute the similarity of John and Lucy (rows 0 and 1):

In [5]:
cosine_similarity(example_matrix[0, :], example_matrix[1, :])

0.5752237416355278

To find John's the nearest neighbors, we need to compute the similarity for all users.

In [6]:
cosine_similarities = [cosine_similarity(example_matrix[0, :], example_matrix[i, :]) 
                       for i in range(example_matrix.shape[0])]
print(cosine_similarities)

[1.0, 0.5752237416355278, 0.6534640392130712, 0.6474892069992044]


We can see that using the Cosine Similarity function, John's rating profile is most similar to Eric and Diane with not much of a difference between these two. Let's see if this observation is the same when we use a different distance measure.

In [7]:
from scipy.spatial import distance

euclidean_distances = [distance.euclidean(example_matrix[0, :], example_matrix[i, :]) 
                       for i in range(example_matrix.shape[0])]
print('Euclidean Distances', euclidean_distances)


Euclidean Distances [0.0, 7.3484692283495345, 5.656854249492381, 5.916079783099616]


Now we can predict John's rating for Die Hard based on his two nearest neighbors, Eric and Diane. Given the similarity of the vector distance, we apply a relatively small weight advantage towards the closest neighbor to John.

In [8]:
rating = (0.6 * example_matrix[2, 2] + 0.4 * example_matrix[3, 2]) / (0.6 + 0.4)
rating

3.8

Luckily, there are a plethora of ready-to-use implementations for different recommendation methods available, so we don't have to implement everything from scratch. Let's use a bigger data set and use the KNN algorithm that sklearn provides.

## KNN with Movielens and Sklearn

The Movielens 100k data set consists of roughly 100,000 ratings of 9000 movies by 600 users. You can download this and other movie rating data sets on the [grouplens website](https://grouplens.org/datasets/movielens/latest/). I provided the two files that we will be working with in the `data` folder.

### Load Movielens

At first, we will load the `ratings.csv` file. We don't need the `timestamp` column which is why we will drop it right away.

In [9]:
ml_ratings = pd.read_csv('data/ratings.csv', usecols=['userId', 'movieId', 'rating'])
ml_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


### Pre-processing
At first, we need to transform the data frame into a rating matrix as seen in our toy example above. We will use an item-based recommendation approach, which is why we set the index to 'movieId' and the columns to 'userId'. Everytime a user has not rated a movie, we would get NaN. In order to feed the data into a KNN algorithm, we need to fill the NaNs with a numerical value, 0 in this case.

In [10]:
ml_matrix = ml_ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)
ml_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


You can see at first glance that the majority of the values in this matrix are 'unknowns'. Scipy offers an efficient way to work with such sparse matrices by compressing them to a csr (compressed sparse row format) matrix.

In [11]:
from scipy.sparse import csr_matrix

ml_csr = csr_matrix(ml_matrix.values)

In addition to the rating data, we also want to include the titles of the movies to be able to manually inspect the nearest neighbors.

In [12]:
movie_df = pd.read_csv('data/movies.csv', usecols=['movieId', 'title'])
movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


We initialize the KNN model by setting the distance measure to cosine similarity and the algorithm to brute-force search. We hold out the first two movies, Toy Story and Jumanji as we want to find the most similar movies to these two later on.

In [13]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=17)
knn.fit(ml_csr[2:, :])

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=17)

Next, we can select one movie and check which movies are the closest neighbors according to the user ratings. The `kneighbors` method takes a movie vector and returns a tuple with two arrays both of shape (1, number of neighbors). The first array is the similarity value, the second the row-indices of the 5 nearest neighbors.

Let's find the nearest neighbors for Toy Story, which is the first movie in our matrix.

In [14]:
distance, indices = knn.kneighbors(ml_csr[0], n_neighbors=20)

In [15]:
distance

array([[0.42739874, 0.4343632 , 0.43573831, 0.44261183, 0.45290409,
        0.45885465, 0.4589107 , 0.46108723, 0.46583124, 0.46961865,
        0.4720232 , 0.47214083, 0.47967548, 0.48196726, 0.48581831,
        0.48775438, 0.49068283, 0.49140738, 0.49145545, 0.49480359]])

In [16]:
indices

array([[2351,  416,  613,  222,  312,  320,  908,  544,  961,  966, 3187,
         504,  121,  255,  895,  813, 1180,   29,  275,   30]])

Let's see what movies these indices correspond to. We first need to split off these movies from the `movie_df` and reset the data frame index before matching them by index. Otherwise the movie names will be off by two rows. Make sure that you only execute this cell once, otherwise it will keep removing the first two rows with every execution.

In [17]:
movie_df = movie_df.drop(index=[0, 1]).reset_index(drop=True)
movie_df.head()

,movieId,title
0,3,Grumpier Old Men (1995)
1,4,Waiting to Exhale (1995)
2,5,Father of the Bride Part II (1995)
3,6,Heat (1995)
4,7,Sabrina (1995)


Let's check the 10 nearest neighbors to Toy Story:

In [18]:
for i in indices:
    print(movie_df['title'][i])

2351                                 'night Mother (1986)
416                                  Jurassic Park (1993)
613                  Independence Day (a.k.a. ID4) (1996)
222             Star Wars: Episode IV - A New Hope (1977)
312                                   Forrest Gump (1994)
320                                 Lion King, The (1994)
908     Once Upon a Time in the West (C'era una volta ...
544                            Mission: Impossible (1996)
961                                           Diva (1981)
966                           Arsenic and Old Lace (1944)
3187            Rififi (Du rififi chez les hommes) (1955)
504                                        Aladdin (1992)
121                                      Apollo 13 (1995)
255                                   Pulp Fiction (1994)
895                 Cheech and Chong's Up in Smoke (1978)
813            Willy Wonka & the Chocolate Factory (1971)
1180                                          Fall (1997)
29            

Now we can repeat the same thing with Jumanji:

In [19]:
similarity, indices = knn.kneighbors(ml_csr[1], n_neighbors=20)
for i in indices:
    print(movie_df['title'][i])

320                                Lion King, The (1994)
434                                Mrs. Doubtfire (1993)
323                                     Mask, The (1994)
416                                 Jurassic Park (1993)
502                                    Home Alone (1990)
481               Nightmare Before Christmas, The (1993)
504                                       Aladdin (1992)
510                          Beauty and the Beast (1991)
16                 Ace Ventura: When Nature Calls (1995)
274                             Santa Clause, The (1994)
129                                        Casper (1995)
273                                      Stargate (1994)
174                                    Waterworld (1995)
335                                     True Lies (1994)
332                                         Speed (1994)
503                                         Ghost (1990)
215    Interview with the Vampire: The Vampire Chroni...
136                    Die Hard

Looking at both lists, you can find similar movies based on a rating matrix. However, there might be better approaches than finding similar movies based on the rating behavior of the users.
Additionally, manually inspecting the movie list is not exactly an objective evaluation method. Let's have a closer look at both in the next section.

## Factorization Methods

While it is also possible to implement a factorization approach using sklearn, be using the [Surprise](http://surpriselib.com/) library for this part of the tutorial to show you an out-of-the-box approach. Surprise is one of many frameworks that provide a selection of recommendation methods for explicit feedback data in a ready-to-use manner. It was heavily inspired by sklearn and uses a similar syntax and provides builtin datasets and evaluation metrics.

We will compare the performance of 2 factorization methods, Singular Value Decomposition (SVD) which became popular when it lead to winning the third place in the Netflix competition, and the similar Non-Negative Matrix Factorization (NMF). You can check out the maths behind both approaches in the documentation ([Link to SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD), [Link to NMF](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF)).

Let's import the functions that we will need for this comparison

In [20]:
from surprise import SVD, NMF
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection.split import train_test_split

Surprise uses their own data format. They provide functions to load external data from a file or a pandas data frame, but lucky for us, the movielens 100k data is one of their builtin data set. Let's load it and split off 20% for testing.

In [21]:
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=.2, random_state=42)

We don't have to worry about any data transformation as Surprise takes care of this. This is why we can initialize and fit the data to the SVD algorithm right away. 

In [22]:
svd_algo = SVD(random_state = 42)
svd_algo.fit(trainset)

The `test` function will create the predictions for our test set. It returns a list with a user ID, item ID, the actual rating, and the estimated rating.

In [23]:
predictions_svd = svd_algo.test(testset)
predictions_svd[:10]

[Prediction(uid='907', iid='143', r_ui=5.0, est=4.611536250297749, details={'was_impossible': False}),
 Prediction(uid='371', iid='210', r_ui=4.0, est=4.37114223755087, details={'was_impossible': False}),
 Prediction(uid='218', iid='42', r_ui=4.0, est=3.7387439108043528, details={'was_impossible': False}),
 Prediction(uid='829', iid='170', r_ui=4.0, est=3.8854605526859616, details={'was_impossible': False}),
 Prediction(uid='733', iid='277', r_ui=1.0, est=2.951243324162181, details={'was_impossible': False}),
 Prediction(uid='363', iid='1512', r_ui=1.0, est=3.1175626582710105, details={'was_impossible': False}),
 Prediction(uid='193', iid='487', r_ui=5.0, est=3.584277708279981, details={'was_impossible': False}),
 Prediction(uid='808', iid='313', r_ui=5.0, est=4.694317679756497, details={'was_impossible': False}),
 Prediction(uid='557', iid='682', r_ui=2.0, est=3.1886298403272635, details={'was_impossible': False}),
 Prediction(uid='774', iid='196', r_ui=3.0, est=2.438638505544496, det

From manual inspection, you can see that a couple of predictions are a bit off while others are very close to the ground truth. Let's put a number to the predictive performance. As we are dealing with explicit data, we can use the `Root Mean Squared Error` (RMSE) to determine the predictive performance. With this metric, a lower value indicates a higher accuracy.

In [24]:
accuracy.rmse(predictions_svd)

RMSE: 0.9352


0.935171451026933

Let's look at another Factorization method, non-negative matrix factorization (NMF). Again, we initialize the model with the default parameters and fit it on the training data.

In [25]:
nmf_algo = NMF(random_state = 42)
nmf_algo.fit(trainset)

Let's inspect the predictions:

In [26]:
predictions_nmf = nmf_algo.test(testset)
predictions_nmf[:10]

[Prediction(uid='907', iid='143', r_ui=5.0, est=4.8146792545930674, details={'was_impossible': False}),
 Prediction(uid='371', iid='210', r_ui=4.0, est=4.19512578582791, details={'was_impossible': False}),
 Prediction(uid='218', iid='42', r_ui=4.0, est=3.525580274112197, details={'was_impossible': False}),
 Prediction(uid='829', iid='170', r_ui=4.0, est=4.131742088580094, details={'was_impossible': False}),
 Prediction(uid='733', iid='277', r_ui=1.0, est=3.709656181355705, details={'was_impossible': False}),
 Prediction(uid='363', iid='1512', r_ui=1.0, est=4.118810905573852, details={'was_impossible': False}),
 Prediction(uid='193', iid='487', r_ui=5.0, est=3.9782946152950114, details={'was_impossible': False}),
 Prediction(uid='808', iid='313', r_ui=5.0, est=4.94266576388802, details={'was_impossible': False}),
 Prediction(uid='557', iid='682', r_ui=2.0, est=2.6266692176433857, details={'was_impossible': False}),
 Prediction(uid='774', iid='196', r_ui=3.0, est=2.2686929898676316, deta

The predictions look very similar to the SVD results. The majority of the estimated ratings are close to the ground truth, but in the middle of the list, the values are pretty off. Let's see if the RSME will confirm this impression that the performance is very similar:

In [27]:
accuracy.rmse(predictions_nmf)

RMSE: 0.9594


0.9594077989819741

Both results only differ by 0.02 and there is most likely room for improvement in both cases by tuning the parameters. However, even with default parameters, the results are not too bad. Let's inspect the actual top-n recommendations for the users to see if this small difference in accuracy has an impact.

The `get_top_n` function was taken from an [example script](https://surprise.readthedocs.io/en/stable/FAQ.html#top-n-recommendations-py) provided by the Surprise library. It iterates through the predictions list and assigns the item IDs and the estimated ratings to the corresponding user ID in a dictionary. The item IDs are then sorted by estimated rating and the resulting list is cut off at `n`.

In [28]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

If we apply this function to both prediction list, we will only get movie IDs in return. To get a better intuition, we match these IDs back with the titles in the `movie_df`.

In [29]:
# SVD Recommendations
top_n = get_top_n(predictions_svd, n=10)
for uid, user_ratings in list(top_n.items())[:5]:
    print('user:', uid)
    print('movie recommendations: ', end='')
    for iid, _ in user_ratings:
        movie = movie_df.loc[movie_df['movieId'] == int(iid)]['title']
        if not movie.empty:
            print(movie.values[0], end=', ')
    print('\n')


user: 907
movie recommendations: Juror, The (1996), Johnny Mnemonic (1995), Courage Under Fire (1996), Flirting With Disaster (1996), Leaving Las Vegas (1995), Net, The (1995), Judge Dredd (1995), Brothers McMullen, The (1995), 

user: 371
movie recommendations: Wild Bill (1995), Hate (Haine, La) (1995), Nine Months (1995), Forget Paris (1995), Kids (1995), Georgia (1995), Carlito's Way (1993), 

user: 218
movie recommendations: White Man's Burden (1995), Dracula: Dead and Loving It (1995), Dead Presidents (1995), Devil in a Blue Dress (1995), Striptease (1996), Seven (a.k.a. Se7en) (1995), True Crime (1996), 

user: 829
movie recommendations: 8 Seconds (1994), Safe (1995), Hackers (1995), Burnt by the Sun (Utomlyonnye solntsem) (1994), Strange Days (1995), From Dusk Till Dawn (1996), Anne Frank Remembered (1995), Girl 6 (1996), 

user: 733
movie recommendations: Nixon (1995), Remains of the Day, The (1993), Sunset Blvd. (a.k.a. Sunset Boulevard) (1950), Escape from New York (1981), Ba

In [30]:
# NMF Recommendations
top_n = get_top_n(predictions_nmf, n=10)
for uid, user_ratings in list(top_n.items())[:5]:
    print('user:', uid)
    print('movie recommendations: ', end='')
    for iid, _ in user_ratings:
        movie = movie_df.loc[movie_df['movieId'] == int(iid)]['title']
        if not movie.empty:
            print(movie.values[0], end=', ')
    print('\n')

user: 907
movie recommendations: Judge Dredd (1995), Larger Than Life (1996), Juror, The (1996), Johnny Mnemonic (1995), Courage Under Fire (1996), Net, The (1995), Flirting With Disaster (1996), Santa Clause, The (1994), 

user: 371
movie recommendations: Kids (1995), Wild Bill (1995), Hate (Haine, La) (1995), Forget Paris (1995), Georgia (1995), Nine Months (1995), Carlito's Way (1993), 

user: 218
movie recommendations: Dracula: Dead and Loving It (1995), White Man's Burden (1995), Devil in a Blue Dress (1995), Dead Presidents (1995), Seven (a.k.a. Se7en) (1995), Striptease (1996), True Crime (1996), 

user: 829
movie recommendations: 8 Seconds (1994), Hackers (1995), Safe (1995), Pie in the Sky (1996), Strange Days (1995), Burnt by the Sun (Utomlyonnye solntsem) (1994), Girl 6 (1996), Circle of Friends (1995), Anne Frank Remembered (1995), 

user: 733
movie recommendations: Nixon (1995), Miracle on 34th Street (1994), Remains of the Day, The (1993), Balto (1995), Escape from New Yo

For better visibility, I copy/pasted the results of the first 3 users into this table:

| UserId   | SVD                      | NMF                    |
|:--------:|:-------------------------|:-----------------------|
| 907      |Juror, The (1996)         |Judge Dredd (1995)      |
|          |Johnny Mnemonic (1995)    |Larger Than Life (1996) |
|          |Courage Under Fire (1996) | Juror, The (1996)      |
|          |Flirting With Disaster (1996)| Johnny Mnemonic (1995)|
|          |Leaving Las Vegas (1995)  | Courage Under Fire (1996)|
|          |Net, The (1995)           | Net, The (1995)      |
|          |Judge Dredd (1995)        | Flirting With Disaster (1996)      |
|          |Brothers McMullen, The (1995)| Santa Clause, The (1994)   |
| 371      |Wild Bill (1995)          |  Kids (1995)     |
|          |Hate (Haine, La) (1995)   |  Wild Bill (1995)     |
|          |Nine Months (1995)        |  Hate (Haine, La) (1995)     |
|          |Forget Paris (1995)       |  Forget Paris (1995)     |
|          |Kids (1995)               |  Georgia (1995)     |
|          |Georgia (1995)            |  Nine Months (1995)     |
|          |Carlito's Way (1993)      |   Carlito's Way (1993)    |
| 218      |White Man's Burden (1995) |  Dracula: Dead and Loving It (1995) |
|          |Dracula: Dead and Loving It (1995)| White Man's Burden (1995)          |
|          |Dead Presidents (1995)    |    Devil in a Blue Dress (1995)          |
|          |Devil in a Blue Dress (1995)|  Dead Presidents (1995)          |
|          |Striptease (1996)         |    Seven (a.k.a. Se7en) (1995)          |
|          |Seven (a.k.a. Se7en) (1995)|    Striptease (1996)         |
|          |True Crime (1996)          |    True Crime (1996)         |


From the table you can see that the order of the list differs slightly, but 80-100% of the movies overlap. From this small sample size, the difference in the RSME does not seem to have a big impact on the recommendations. 

Overall, Surprise provides an easy to use approach to building a recommender system. If I need more control over what is happening, it is better to implement your recommendation system with sklearn, scipy and the likes as Surprise behaves like a black box.